In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.table import Table
from my_utilities import *

In [ ]:
def load_minijpas_jnep(cat_list=['minijpas', 'jnep'], apply_flags=True):
    # If selection, return the valuable items for visual_inspection.py only
    pm_flx = np.array([]).reshape(60, 0)
    pm_err = np.array([]).reshape(60, 0)
    tile_id = np.array([])
    parallax_sn = np.array([])
    pmra_sn = np.array([])
    pmdec_sn = np.array([])
    starprob = np.array([])
    starlhood = np.array([])
    spCl = np.array([])
    zsp = np.array([])
    photoz = np.array([])
    photoz_odds = np.array([])
    photoz_chi_best = np.array([])
    x_im = np.array([])
    y_im = np.array([])
    RA = np.array([])
    DEC = np.array([])
    number = np.array([])

    N_minijpas = 0
    split_converter = lambda s: np.array(s.split()).astype(float)
    sum_flags = lambda s: np.sum(np.array(s.split()).astype(float))

    for name in cat_list:
        cat = pd.read_csv(f'../LAEs/csv/{name}.Flambda_aper3_photoz_gaia_3.csv', sep=',', header=1,
            converters={0: int, 1: int, 2: split_converter, 3: split_converter, 4: sum_flags,
            5: sum_flags})

        cat = cat[np.array([len(x) for x in cat['FLUX_APER_3_0']]) != 0] # Drop bad rows due to bad query
        if apply_flags:
            cat = cat[(cat.FLAGS == 0) & (cat.MASK_FLAGS == 0)] # Drop flagged
        cat = cat.reset_index()

        tile_id_i = cat['TILE_ID'].to_numpy()

        parallax_i = cat['parallax'].to_numpy() / cat['parallax_error'].to_numpy()
        pmra_i = cat['pmra'].to_numpy() / cat['pmra_error'].to_numpy()
        pmdec_i = cat['pmdec'].to_numpy() / cat['pmdec_error'].to_numpy()

        pm_flx_i = np.stack(cat['FLUX_APER_3_0'].to_numpy()).T * 1e-19
        pm_err_i = np.stack(cat['FLUX_RELERR_APER_3_0'].to_numpy()).T * pm_flx_i

        if name == 'minijpas':
            N_minijpas = pm_flx_i.shape[1]
        
        starprob_i = cat['morph_prob_star']
        starlhood_i = cat['morph_lhood_star']

        RA_i = cat['ALPHA_J2000']
        DEC_i = cat['DELTA_J2000']

        pm_err_i = (pm_err_i ** 2 + Zero_point_error(cat['TILE_ID'], name) ** 2) ** 0.5

        spCl_i = cat['spCl']
        zsp_i = cat['zsp']

        photoz_i = cat['PHOTOZ']
        photoz_odds_i = cat['ODDS']
        photoz_chi_best_i = cat['CHI_BEST']

        x_im_i = cat['X_IMAGE']
        y_im_i = cat['Y_IMAGE']

        number_i = cat['NUMBER']

        pm_flx = np.hstack((pm_flx, pm_flx_i))
        pm_err = np.hstack((pm_err, pm_err_i))
        tile_id = np.concatenate((tile_id, tile_id_i))
        pmra_sn = np.concatenate((pmra_sn, pmra_i))
        pmdec_sn = np.concatenate((pmdec_sn, pmdec_i))
        parallax_sn = np.concatenate((parallax_sn, parallax_i))
        starprob = np.concatenate((starprob, starprob_i))
        starlhood = np.concatenate((starlhood, starlhood_i))
        spCl = np.concatenate((spCl, spCl_i))
        zsp = np.concatenate((zsp, zsp_i))
        photoz = np.concatenate((photoz, photoz_i))
        photoz_odds = np.concatenate((photoz_odds, photoz_odds_i))
        photoz_chi_best = np.concatenate((photoz_chi_best, photoz_chi_best_i))
        x_im = np.concatenate((x_im, x_im_i))
        y_im = np.concatenate((y_im, y_im_i))
        RA = np.concatenate((RA, RA_i))
        DEC = np.concatenate((DEC, DEC_i))
        number = np.concatenate((number, number_i))

    return pm_flx, pm_err, tile_id, pmra_sn, pmdec_sn, parallax_sn, starprob, starlhood,\
        spCl, zsp, photoz, photoz_chi_best, photoz_odds, N_minijpas, x_im, y_im, RA, DEC

In [ ]:
dirname = '/home/alberto/almacen/Stack_catalogs'
cat_name = f'cat_Unique_ALL_FIELDS_Pband_gri.Table.fits'
    
cat = Table.read(f'{dirname}/{cat_name}')
cat = cat[cat['FLAGS'] == 0]
ra_stack = cat['ALPHA_J2000']
dec_stack = cat['DELTA_J2000']

ra_mj, dec_mj = load_minijpas_jnep(['minijpas'], apply_flags=True)[-2:]

In [ ]:
N_src_stack = len(cat)
ang_dist = np.zeros_like(dec_stack)
mj_min_ang_dist = np.zeros_like(ra_mj)
for src in range(N_src_stack):
    ang_dist_Arr = ((ra_mj - ra_stack[src]) ** 2 + (dec_mj - dec_stack[src]) ** 2) ** 0.5
    ang_dist[src] = ang_dist_Arr.min()

for src in range(len(ra_mj)):
    mj_min_ang_dist[src] = np.min(((ra_mj[src] - ra_stack) ** 2 + (dec_mj[src] - dec_stack) ** 2) ** 0.5)

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))

bins = np.linspace(0, 100, 200)
ax.hist(mj_min_ang_dist * 3600, bins, histtype='step', label='r min distance to stack')
ax.hist(ang_dist * 3600, bins, histtype='step', label='Stack min distance to r')
ax.axvline(3, c='r')
ax.set_yscale('log')
ax.legend()
ax.set_xlabel('Angular distance (arcsec)')
ax.set_ylabel('N')

print(f'Stack sources with 3\" xmatch: {sum(ang_dist < 3 / 3600)} / {len(ang_dist)}')
print(f'r sources with 3\" xmatch: {sum(mj_min_ang_dist < 3 / 3600)} / {len(mj_min_ang_dist)}')

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))

ax.plot(ra_mj, dec_mj, ls='', marker='.', markersize=1, c='C0')
ax.plot(ra_stack, dec_stack, ls='', marker='.', markersize=1, c='C1')
ax.scatter([0], [0], color='C0', label='r catalog')
ax.scatter([0], [0], color='C1', label='Stack')

ax.set_ylim(51.75, 53.75)
ax.set_xlim(213, 216.1)
ax.legend()

plt.show()